In [1]:
import re

In [2]:
query = """
-- start query 6 in stream 0 using template query80.tpl
with ssr as
(select  s_store_id as store_id,
sum(ss_ext_sales_price) as sales,
sum(coalesce(sr_return_amt, 0)) as returns,
sum(ss_net_profit - coalesce(sr_net_loss, 0)) as profit
from store_sales left outer join store_returns on
(ss_item_sk = sr_item_sk and ss_ticket_number = sr_ticket_number),
date_dim,
store,
item,
promotion
where ss_sold_date_sk = d_date_sk
and d_date between cast('2002-08-04' as date)
and (cast('2002-08-04' as date) +  30 days)
and ss_store_sk = s_store_sk
and ss_item_sk = i_item_sk
and i_current_price > 50
and ss_promo_sk = p_promo_sk
and p_channel_tv = 'N'
group by s_store_id)
,
csr as
(select  cp_catalog_page_id as catalog_page_id,
sum(cs_ext_sales_price) as sales,
sum(coalesce(cr_return_amount, 0)) as returns,
sum(cs_net_profit - coalesce(cr_net_loss, 0)) as profit
from catalog_sales left outer join catalog_returns on
(cs_item_sk = cr_item_sk and cs_order_number = cr_order_number),
date_dim,
catalog_page,
item,
promotion
where cs_sold_date_sk = d_date_sk
and d_date between cast('2002-08-04' as date)
and (cast('2002-08-04' as date) +  30 days)
and cs_catalog_page_sk = cp_catalog_page_sk
and cs_item_sk = i_item_sk
and i_current_price > 50
and cs_promo_sk = p_promo_sk
and p_channel_tv = 'N'
group by cp_catalog_page_id)
,
wsr as
(select  web_site_id,
sum(ws_ext_sales_price) as sales,
sum(coalesce(wr_return_amt, 0)) as returns,
sum(ws_net_profit - coalesce(wr_net_loss, 0)) as profit
from web_sales left outer join web_returns on
(ws_item_sk = wr_item_sk and ws_order_number = wr_order_number),
date_dim,
web_site,
item,
promotion
where ws_sold_date_sk = d_date_sk
and d_date between cast('2002-08-04' as date)
and (cast('2002-08-04' as date) +  30 days)
and ws_web_site_sk = web_site_sk
and ws_item_sk = i_item_sk
and i_current_price > 50
and ws_promo_sk = p_promo_sk
and p_channel_tv = 'N'
group by web_site_id)
select  channel
, id
, sum(sales) as sales
, sum(returns) as returns
, sum(profit) as profit
from
(select 'store channel' as channel
, 'store' || store_id as id
, sales
, returns
, profit
from   ssr
union all
select 'catalog channel' as channel
, 'catalog_page' || catalog_page_id as id
, sales
, returns
, profit
from  csr
union all
select 'web channel' as channel
, 'web_site' || web_site_id as id
, sales
, returns
, profit
from   wsr
) x
group by rollup (channel, id)
order by channel
,id
limit 100;

-- end query 6 in stream 0 using template query80.tpl
"""

In [6]:
lines = query.split("\n")

In [7]:
len(lines)

98

In [36]:
regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar"s'
myfile2 = regex.sub(lambda m: m.group().replace('"',"%",1), myfile)
print(myfile2)

foo%s bar%s


In [21]:
reg = re.compile(r"\d+ days")

In [75]:
txt = "and (cast('2002-08-04' as date) -  30 days) and (cast('2002-08-04' as date) + 30 days)"

In [89]:
txt = "and (cast('2002-08-04' as date) -  30 days) and (cast('2002-08-04' as date) + 30 days)"
r = re.compile(r"(\+|\-)\s*(?P<num>\d+)\s*(?P<day>days\))", re.S)
txt = r.sub(lambda m: m.group().replace(m.group("num"), "INTERVAL {}".format(m.group("num")), 1).replace(m.group("day"), "day)", 1), txt)
txt

"and (cast('2002-08-04' as date) -  INTERVAL 30 day) and (cast('2002-08-04' as date) + INTERVAL 30 day)"

In [88]:
txt = 'count(distinct ws_order_number) as "order count"'
r = re.compile(r"(as)\s*(?P<p1>\")\s*(?P<text>[^]]+)\s*(?P<p2>\")", re.S)
txt = r.sub(lambda m: m.group().replace(m.group("text"), "{}".format("_".join(m.group("text").split(" "))), 1).replace(m.group("p1"), "", 1).replace(m.group("p2"), "", 1), txt)
txt

'count(distinct ws_order_number) as order_count'

In [72]:
# t = "{t} {t}".format(t=txt)
t = txt
re.findall(r"(\+|\-)\s*(?P<num>\d+ days\))", t)
re.

SyntaxError: invalid syntax (293274236.py, line 4)

In [8]:
def modify_line(line):

    # +/- <Digit> days) -> INTERVAL <Digit> day)
    interval_regex = re.compile(r"(\+|\-)\s*(?P<num>\d+)\s*(?P<day>days\))", re.S)
    line = interval_regex.sub(lambda m: m.group().replace(m.group("num"), "INTERVAL {}".format(m.group("num")), 1).replace(m.group("day"), "day)", 1), line)
    
    # as "<some text>" -> as some_text
    as_regex = re.compile(r"(as)\s*(?P<p1>\")\s*(?P<text>[^]]+)\s*(?P<p2>\")", re.S)
    line = as_regex.sub(lambda m: m.group().replace(m.group("text"), "{}".format("_".join(m.group("text").split(" "))), 1).replace(m.group("p1"), "", 1).replace(m.group("p2"), "", 1), line)
    
    return line

In [9]:
new_query = ''
for line in lines:
    new_query += "\n{}".format(modify_line(line))
print(new_query)



-- start query 6 in stream 0 using template query80.tpl
with ssr as
(select  s_store_id as store_id,
sum(ss_ext_sales_price) as sales,
sum(coalesce(sr_return_amt, 0)) as returns,
sum(ss_net_profit - coalesce(sr_net_loss, 0)) as profit
from store_sales left outer join store_returns on
(ss_item_sk = sr_item_sk and ss_ticket_number = sr_ticket_number),
date_dim,
store,
item,
promotion
where ss_sold_date_sk = d_date_sk
and d_date between cast('2002-08-04' as date)
and (cast('2002-08-04' as date) +  30 days)
and ss_store_sk = s_store_sk
and ss_item_sk = i_item_sk
and i_current_price > 50
and ss_promo_sk = p_promo_sk
and p_channel_tv = 'N'
group by s_store_id)
,
csr as
(select  cp_catalog_page_id as catalog_page_id,
sum(cs_ext_sales_price) as sales,
sum(coalesce(cr_return_amount, 0)) as returns,
sum(cs_net_profit - coalesce(cr_net_loss, 0)) as profit
from catalog_sales left outer join catalog_returns on
(cs_item_sk = cr_item_sk and cs_order_number = cr_order_number),
date_dim,
catalog_page